# WebSocket Proxy Connection Debug

This notebook investigates the "Not a valid xpra server?" error occurring when the Xpra HTML5 client attempts to auto-connect through a JupyterHub proxy.

## Problem Analysis
- ✅ Firefox server process starts successfully on port 53125
- ✅ WebSocket connection opens successfully  
- ❌ Xpra client immediately disconnects with "Not a valid xpra server?"
- 🔍 **Hypothesis**: The underlying xpra-html5-client bypasses our ProxyCompatibleWebSocket override during protocol handshake

## Investigation Plan
1. Test WebSocket connections with different protocols
2. Verify proxy header configuration
3. Analyze Xpra server response format
4. Fix auto-connection WebSocket override issue

In [1]:
# Import Required Libraries
import asyncio
import websockets
import logging
import json
import time
import socket
from contextlib import asynccontextmanager
from typing import Dict, Any, Optional, List

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🔧 Imported libraries for WebSocket debugging")

🔧 Imported libraries for WebSocket debugging


In [2]:
# Check if Xpra Server is Running
def check_port_status(host: str, port: int) -> Dict[str, Any]:
    """Check if a port is open and responding"""
    try:
        # Test TCP connection
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        
        tcp_open = result == 0
        
        return {
            "host": host,
            "port": port,
            "tcp_open": tcp_open,
            "status": "open" if tcp_open else "closed"
        }
    except Exception as e:
        return {
            "host": host,
            "port": port,
            "tcp_open": False,
            "status": "error",
            "error": str(e)
        }

# Check the Xpra server port from the logs
xpra_port = 53125
host = "192.168.7.10"

port_status = check_port_status(host, xpra_port)
print(f"🔍 Port status: {json.dumps(port_status, indent=2)}")

if port_status["tcp_open"]:
    print("✅ TCP connection to Xpra server is possible")
else:
    print("❌ TCP connection to Xpra server failed")

🔍 Port status: {
  "host": "192.168.7.10",
  "port": 53125,
  "tcp_open": true,
  "status": "open"
}
✅ TCP connection to Xpra server is possible


In [ ]:
# Test WebSocket Connection with Different Protocols
async def test_websocket_connection(uri: str, subprotocols: Optional[List[str]] = None) -> Dict[str, Any]:
    """Test WebSocket connection with specified subprotocols"""
    try:
        print(f"🔗 Testing WebSocket: {uri}")
        if subprotocols:
            print(f"📡 Subprotocols: {subprotocols}")
        
        # Try to connect
        async with websockets.connect(
            uri, 
            subprotocols=subprotocols,
            timeout=10
        ) as websocket:
            print(f"✅ WebSocket connected successfully")
            print(f"📋 Selected subprotocol: {websocket.subprotocol}")
            print(f"📋 Remote address: {websocket.remote_address}")
            
            # Try to receive a message (Xpra should send hello)
            try:
                # Set a short timeout for message receive
                message = await asyncio.wait_for(websocket.recv(), timeout=5.0)
                print(f"📨 Received message type: {type(message)}")
                print(f"📨 Message length: {len(message) if hasattr(message, '__len__') else 'N/A'}")
                
                # Try to decode if it's binary
                if isinstance(message, bytes):
                    print(f"📨 Binary message preview: {message[:50]}...")
                else:
                    print(f"📨 Text message preview: {message[:200]}...")
                    
                return {
                    "status": "success",
                    "subprotocol": websocket.subprotocol,
                    "message_received": True,
                    "message_type": type(message).__name__,
                    "message_length": len(message) if hasattr(message, '__len__') else None
                }
                
            except asyncio.TimeoutError:
                print("⏰ No message received within timeout")
                return {
                    "status": "connected_no_message",
                    "subprotocol": websocket.subprotocol,
                    "message_received": False
                }
                
    except websockets.exceptions.InvalidHandshake as e:
        print(f"❌ Invalid handshake: {e}")
        return {"status": "invalid_handshake", "error": str(e)}
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return {"status": "failed", "error": str(e)}

# Test different protocol combinations
test_cases = [
    {"uri": f"ws://{host}:{xpra_port}/", "subprotocols": None},
    {"uri": f"ws://{host}:{xpra_port}/", "subprotocols": ["binary"]},
    {"uri": f"ws://{host}:{xpra_port}/", "subprotocols": ["xpra"]},
    {"uri": f"ws://{host}:{xpra_port}/", "subprotocols": ["binary", "xpra"]},
]

print("🧪 Testing WebSocket connections...\n")

for i, test_case in enumerate(test_cases, 1):
    print(f"--- Test {i} ---")
    result = await test_websocket_connection(**test_case)
    print(f"Result: {json.dumps(result, indent=2)}")
    print()